# Project 2

In [1]:
import torch
import math
torch.set_grad_enabled(False)

In [2]:
class Module(object):
    def __init__(self):
        self.parameters = []
        
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return self.parameters

In [3]:
class Linear(Module):
    def __init__(self, in_features, out_features, bias=False):
        init_range = math.sqrt(in_features)
        self.weights = torch.Tensor(in_features, out_features).uniform_(-init_range, init_range)
        if bias:
            self.bias = torch.Tensor(out_features).uniform_(-init_range, init_range)
        else:
            self.bias = None
        
    def forward(self, input_):
        if self.bias != None:
            return torch.addmm(self.bias, input_, self.weights)
        else:
            return input_.matmul(self.weights)
        
    def backward(self):
        pass